In [1]:
# Install the latest version of crossdeploy
! pip install --upgrade crossdeploy

# Node is installed by default in Watson Studio, JupyterLab and JupyterNotebook environments. 
# However, in case your environment does not have node installed, it can be installed with the below command.
# ! pip install "nodejs-bin[cmd]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 MB 28.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 2.12.0
    Uninstalling typeguard-2.12.0:
      Successfully uninstalled typeguard-2.12.0


# Example 2: Build, promote and deploy a model

In this example, we will build, promote and deploy a simple SKLearn pipeline model.

# Imports

In [2]:
import pandas as pd

from crossdeploy.utils import utils
from crossdeploy.crossdeploy import CrossDeploy

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer

import ibm_watson_machine_learning

# Define variables

In [3]:
CPD_API_KEY = "xxx" # or CPD_PASSWORD = "xxx"
CPD_USERNAME = "admin"
CPD_URL = "https://cpd-cpd.itzroks-550003aw18-bwcq3a-6ccd7f378ae819553d37d5f2ee142bd6-0000.au-syd.containers.appdomain.cloud"
PROJECT_ID = "75c938f9-cc5f-45a9-b44f-3f1b51a69516"
SPACE_ID = "c80c0f20-ce3e-45fe-a0ae-e8012abffd81"

wml_credentials = {"username": CPD_USERNAME, "apikey": CPD_API_KEY, "url": CPD_URL, "instance_id": "openshift", "version": "4.6"}
wml_client = ibm_watson_machine_learning.APIClient(wml_credentials)

# Import data and preprocessing step

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/crossdeploy-io/crossdeploy-examples/main/data/mortgage-default.csv")

label_column = "MortgageDefault"
y = df[label_column]
X = df.drop(label_column, axis=1)

ct = make_column_transformer(
    (OneHotEncoder(), make_column_selector(dtype_include=object)),
    remainder="passthrough"
)

# Build model

In [10]:
model = Pipeline(steps=[
    ("transform", ct), 
    ("clf", RandomForestClassifier(n_estimators=3, max_depth=3, random_state=12345))
])
model.fit(X, y)
model.score(X, y)

0.639618138424821

# Save, promote and deploy model

Making changes, `MODEL_NAME` or `DEPLOYMENT_NAME`, will result in an update as there is no material impact to the underlying asset.

However, when the model is trained with new hyperparameters, this will result in destroy, the current version of the model and its promoted asset, creating and promoting a new model and updating the underlying asset of the existing deployment.

In [11]:
MODEL_NAME = "mortgage-model-rf"
DEPLOYMENT_NAME = "mortgage-model-rf"

flow = CrossDeploy()

flow.ibm.Provider(url=CPD_URL, username=CPD_USERNAME, api_key=CPD_API_KEY) # or password=CPD_PASSWORD

model_rf = flow.ibm.Model(model)(
    id = "mortgage-model-rf",
    name = MODEL_NAME,
    project_id = PROJECT_ID,
)

promoted_model_rf = flow.ibm.PromotedModel(model_rf)(
    id = "mortgage-model-rf-promoted",
    project_id = PROJECT_ID,
    space_id = SPACE_ID,
    asset_id = model_rf.id,
)

deployment = flow.ibm.Deployment(
    id = "mortgage-deployment-rf",
    name = DEPLOYMENT_NAME,
    space_id = SPACE_ID,
    asset = promoted_model_rf.id,
    online = True,
)

flow.apply()

deployment_state = utils.get_resource_by_name(deployment.friendly_unique_id)

print()
print(f"Deployment ID: {deployment_state['id']}")

Applying ...
ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80: Refreshing state... [id=624f7965-7862-4412-a82c-8c2144ed7704]
ibmcpd_model.crossdeploy_mortgagemodelrfpromoted_9850AEA8: Refreshing state... [id=cf64ea0e-4ab0-4a0e-aa80-826009520171]
ibmcpd_deployment.crossdeploy_mortgagedeploymentrf_58AD9F47: Refreshing state... [id=c67283fc-693b-4f92-8042-6fb481e2f48d]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  ~ update in-place
+/- create replacement and then destroy

Terraform will perform the following actions:

  # ibmcpd_deployment.crossdeploy_mortgagedeploymentrf_58AD9F47 will be updated in-place
  ~ resource "ibmcpd_deployment" "crossdeploy_mortgagedeploymentrf_58AD9F47" {
      ~ asset    = "cf64ea0e-4ab0-4a0e-aa80-826009520171" -> (known after apply)
        id       = "c67283fc-693b-4f92-8042-6fb481e2f48d"
        name     = "mortgage-model-rf"
      ~ url      = "https://cpd-cpd.

# Score model

In [13]:
wml_client.set.default_space(SPACE_ID)

deployment_state = utils.get_resource_by_name(deployment.friendly_unique_id)

deployment_id = deployment_state["id"]

payload = {"input_data": [{"values": X.head()}]}

wml_client.deployments.score(deployment_id, payload)


{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['NO', [1.0, 0.0]],
    ['YES', [0.3333333333333333, 0.6666666666666666]],
    ['NO', [1.0, 0.0]],
    ['NO', [0.6666666666666666, 0.3333333333333333]],
    ['NO', [1.0, 0.0]]]}]}

# Clean up

In [12]:
flow.destroy()

Destroying ...
ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80: Refreshing state... [id=86131972-189b-49a1-9a8b-4bbc3afb7589]
ibmcpd_model.crossdeploy_mortgagemodelrfpromoted_9850AEA8: Refreshing state... [id=3f6fea77-5209-4791-b278-9dc2a6502ba5]
ibmcpd_deployment.crossdeploy_mortgagedeploymentrf_58AD9F47: Refreshing state... [id=c67283fc-693b-4f92-8042-6fb481e2f48d]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # ibmcpd_deployment.crossdeploy_mortgagedeploymentrf_58AD9F47 will be destroyed
  - resource "ibmcpd_deployment" "crossdeploy_mortgagedeploymentrf_58AD9F47" {
      - asset    = "3f6fea77-5209-4791-b278-9dc2a6502ba5" -> null
      - id       = "c67283fc-693b-4f92-8042-6fb481e2f48d" -> null
      - name     = "mortgage-model-rf" -> null
      - online   = true -> null
      - space_id = "c80c0f20-ce3e-45fe-a0ae-e8012abffd81